In [1]:
#import des librairies
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4040 --packages com.datastax.spark:spark-cassandra-connector_2.11:2.3.2 pyspark-shell'
import time

In [2]:
#initiation de context de spark cassandra
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row
from pyspark.streaming import StreamingContext
from pyspark.sql.functions import desc

In [3]:
conf = SparkConf() \
    .setAppName("csv to cassandra") \
    .setMaster("local[2]") \
    .set("spark.cassandra.connection.host", "127.0.0.1")
sc = SparkContext(conf=conf) 
sqlContext=SQLContext(sc)

In [4]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

In [5]:
socket_stream = ssc.socketTextStream("localhost", 5555)

In [6]:
lines = socket_stream.window( 10 )

In [7]:
from collections import namedtuple
fields = ("id","source", "user_id","user_name","user_followers_count" )
Tweet = namedtuple( 'Tweet', fields )

In [8]:
#connexion à cassandra
def saveToCassandra(rows):
    if not rows.isEmpty(): 
        print(rows[1])
        sqlContext.createDataFrame(rows).write\
        .format("org.apache.spark.sql.cassandra")\
        .mode('append')\
        .options(table="tweet", keyspace="ereputation")\
        .save()

In [9]:
( lines.flatMap( lambda text: text.split( "|" ) )
  
 .map( lambda rec: Tweet( rec[0] ) )
 .foreachRDD( lambda rdd: saveToCassandra ) )

In [10]:
# Start the computation
ssc.start()

In [11]:
#ssc.stop()